In [1]:
%matplotlib widget
from pathlib import Path
import sys
import os
import warnings

import matplotlib.pyplot as plt
plt.ioff()
    
from micro_helpers import pims
from smfret_analysis import Tracker, print_info

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
print_info()

smFRET analysis software version 2.1
(git revision 2.1-56-g70f9081)
Output version 11
Using sdt-python version 14.1


In [4]:
load = False

if not load:
    # Start fresh
    data_dir = "/media/lukas/LS-ext-blue/Data/Force sensor FRET/2018-11-21"
    # donor origin, acceptor origin, size
    tr = Tracker((201, 0), (0, 0), (190, 111), "ce" + "da" * 300 + "c", data_dir=data_dir)
    tr.add_dataset("DPPC sensor cells",
                   Path("^DPPC_sensor-J4", r"cells-\d{2}_(\d{3})_.SPE"), cells=True)
    tr.add_dataset("DPPC sensor no-cells",
                   Path("^DPPC_sensor-J4", r"no-cells_(\d{3})_.SPE"), cells=False)
    tr.add_dataset("POPC sensor cells",
                   Path("^POPC_sensor-J4", r"cells-\d{2}_(\d{3})_.SPE"), cells=True)
    tr.add_dataset("POPC sensor no-cells",
                   Path("^POPC_sensor-J4", r"no-cells_(\d{3})_.SPE"), cells=False)
    tr.add_dataset("DPPC control cells",
                   Path("^DPPC_ctrl-J4", r"cells-\d{2}_(\d{3})_.SPE"), cells=True)
    tr.add_dataset("DPPC control no-cells",
                   Path("^DPPC_ctrl-J4", r"no-cells_(\d{3})_.SPE"), cells=False)
    tr.add_dataset("POPC control cells",
                   Path("^POPC_ctrl-J4", r"cells-\d{2}_(\d{3})_.SPE"), cells=True)
    tr.add_dataset("POPC control no-cells",
                   Path("^POPC_ctrl-J4", r"no-cells_(\d{3})_.SPE"), cells=False)
else:
    # load
    tr = Tracker.load()

In [5]:
tr.set_bead_loc_opts(Path("beads", "beads_\d+_.SPE"))

Locator(children=(HBox(children=(VBox(children=(VBox(children=(Select(description='files', layout=Layout(width…

In [9]:
tr.make_chromatic()

Label(value='Starting…')

FigureCanvasNbAgg()

In [10]:
tr.set_loc_opts("donor")

In [11]:
tr.set_loc_opts("acceptor")

In [12]:
tr.locate()

Label(value='Starting…')

In [5]:
tr.track(feat_radius=4, bg_frame=3, link_radius=5, link_mem=3, min_length=4, bg_estimator="mean")

Label(value='Starting…')

In [6]:
tr.extract_cell_images()
tr.make_flatfield("donor", Path("profile", r"green_.*"), bg=200, smooth_sigma=10)
tr.make_flatfield("acceptor", Path("profile", r"red_.*"), bg=200, smooth_sigma=10)

In [7]:
tr.save()

# Extract experiment times for paper

In [8]:
from collections import OrderedDict
from sdt import io


def get_experiment_times(tracker, key):
    times = OrderedDict()
    for n, f in tracker.sources[key]["files"].items():
        times[n] = f.metadata["DateTime"]
    with open("experiment_times.yaml", "w") as f:
        io.yaml.safe_dump(times, f)
        
get_experiment_times(tr, "DPPC sensor cells")